In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

%load_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot as plt
from glob import glob

import enterprise
from enterprise.pulsar import Pulsar
from enterprise.signals import parameter,white_signals,gp_signals,signal_base
from enterprise.signals.utils import powerlaw

from enterprise_extensions import blocks
from enterprise_extensions.model_utils import get_tspan

from la_forge.core import Core

import sys
sys.path.append('../')

from defiant import OptimalStatistic
from defiant import utils

In [2]:
datadir = enterprise.__path__[0] + '/datafiles/mdc_open1/'
parfiles = sorted(glob(datadir + '/*.par'))
timfiles = sorted(glob(datadir + '/*.tim'))

psrs = [Pulsar(par,tim) for par,tim in zip(parfiles,timfiles)]

inj_params = {'gw_log10_A':np.log10(5e-14)}

[tempo2Util.C:396] Warning: [TIM1] Please place MODE flags in the parameter file 
[tempo2Util.C:401] Warning: [DUP1] duplicated warnings have been suppressed.


In [69]:
Tspan = get_tspan(psrs)

efac = parameter.Constant(1.0)
ef = white_signals.MeasurementNoise(efac=efac)

curn = blocks.common_red_noise_block(psd='powerlaw',Tspan=Tspan,components=10,
                                     gamma_val=13./3.,logmin=-18,logmax=-12,name='gw')

tm = gp_signals.TimingModel(use_svd=True)
m_tm = gp_signals.MarginalizingTimingModel(use_svd=True)

m_model = m_tm + ef + curn
model = tm + ef + curn

pta = signal_base.PTA([model(psr) for psr in psrs])
m_pta = signal_base.PTA([m_model(psr) for psr in psrs])

In [70]:
OS_obj = OptimalStatistic(psrs,pta=pta,gwb_name='gw')
m_OS_obj = OptimalStatistic(psrs,pta=m_pta,gwb_name='gw')

/Users/gersbaka/Documents/Projects/defiant/notebooks/../defiant/core.py:168: UserWarning: No MCMC samples were given! Set these later or supply them when computing the OS.
  warn(msg)


In [100]:
X,Z = OS_obj._compute_XZ(inj_params)
m_X,m_Z = m_OS_obj._compute_XZ(inj_params)

In [101]:
x_diff = np.abs((X-m_X)/X)
z_diff = np.abs((Z-m_Z)/Z)

print(np.max(x_diff),np.max(z_diff))

2.0128675898630554e-10 2.971215049868253e-08


In [103]:
xi,rho,sig,C,A2,A2s = OS_obj.compute_OS(inj_params,pair_covariance=False)
print(np.sqrt(A2),A2/A2s)

xi,rho,sig,C,A2,A2s = m_OS_obj.compute_OS(inj_params,pair_covariance=False)
print(np.sqrt(A2),A2/A2s)

5.010145033906853e-14 13.149346699665019
5.010145033905882e-14 13.149346699660917


In [102]:
%timeit OS_obj.compute_OS(inj_params,pair_covariance=False)
%timeit m_OS_obj.compute_OS(inj_params,pair_covariance=False)

8.96 ms ± 825 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.19 ms ± 586 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
